In [1]:
import pandas as pd
from sklearn import datasets
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).
2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по
всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки
известны на момент обучения, и поэтому можно ими пользоваться
при обучении алгоритма.
3. Подберите минимальный лучший параметр C из множества [10^−5, 10^−4, ...10^4, 10^5]
для SVM с линейным ядром (kernel=’linear’) при помощи кроссвалидации по 5 блокам. Укажите параметр random_state=241 и
для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
4. Обучите SVM по всей выборке с лучшим параметром C, найденным
на предыдущем шаге.
5. Найдите 10 слов с наибольшим по модулю весом. Они являются
ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.


In [3]:
newsgroups = datasets.fetch_20newsgroups(
    subset='all',
    categories=['alt.atheism', 'sci.space']
)

In [ ]:
newsgroups.data[0:3]

['From: 9051467f@levels.unisa.edu.au (The Desert Brat)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: Cured, discharged\nLines: 24\n\nIn article <1pa0f4INNpit@gap.caltech.edu>, keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n> But really, are you threatened by the motto, or by the people that use it?\n\nEvery time somone writes something and says it is merely describing the norm,\nit is infact re-inforcing that norm upon those programmed not to think for\nthemselves. The motto is dangerous in itself, it tells the world that every\n*true* American is god-fearing, and puts down those who do not fear gods. It\ndoesn\'t need anyone to make it dangerous, it does a good job itself by just\nexisting on your currency.\n\n> keith\n\nThe Desert Brat\n-- \nJohn J McVey, Elc&Eltnc Eng, Whyalla, Uni S Australia,    ________\n9051467f@levels.unisa.edu.au      T.S.A.K.C.            \\/Darwin o\\\nFor replies, mail to whjjm@wh.whyalla.unisa.edu.au      /\\________/\nDisclaimer

In [ ]:
newsgroups.target[0:3]

array([0, 0, 1])

In [4]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(newsgroups.data)

In [ ]:
grid = {'C' : np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(features, newsgroups.target)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [ ]:
for a in gs.cv_results_:
    print(a)

mean_fit_time
std_fit_time
mean_score_time
std_score_time
param_C
params
split0_test_score
split1_test_score
split2_test_score
split3_test_score
split4_test_score
mean_test_score
std_test_score
rank_test_score


In [ ]:
for i in ['mean_test_score', 'params']:

            print(i,"  ", gs.cv_results_[i])

mean_test_score    [0.55263603 0.55263603 0.55263603 0.55263603 0.95016353 0.99328044
 0.99328044 0.99328044 0.99328044 0.99328044 0.99328044]
params    [{'C': 1e-05}, {'C': 0.0001}, {'C': 0.001}, {'C': 0.01}, {'C': 0.1}, {'C': 1.0}, {'C': 10.0}, {'C': 100.0}, {'C': 1000.0}, {'C': 10000.0}, {'C': 100000.0}]


лучший параметр C = 1000

In [5]:
clf_v2 = SVC(kernel='linear', random_state=241, C=1000.0)
clf_v2.fit(features, newsgroups.target)

SVC(C=1000.0, kernel='linear', random_state=241)

In [ ]:
clf_v2.coef_[0:10]

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18483 stored elements in Compressed Sparse Row format>

In [6]:
dataframe = pd.DataFrame(clf_v2.coef_.toarray().T, columns=['coeff'])
print(dataframe)

          coeff
0      0.273658
1     -0.107055
2      0.000000
3      0.000000
4      0.000000
...         ...
28377  0.009973
28378  0.009973
28379  0.019946
28380  0.059502
28381 -0.003014

[28382 rows x 1 columns]


In [7]:
coef_abs_lsrgest = dataframe.abs().nlargest(10, 'coeff')
print(coef_abs_lsrgest)

          coeff
24019  2.720304
12871  1.971924
5088   1.299935
5093   1.257272
17802  1.248378
23673  1.195954
21850  1.155796
5776   1.126725
15606  1.107114
18430  1.089924


In [8]:
feature_mapping = pd.DataFrame(vectorizer.get_feature_names_out(), columns=['words'])
print(feature_mapping)

        words
0          00
1         000
2        0000
3       00000
4      000000
...       ...
28377  zwakke
28378   zware
28379  zwarte
28380   zwork
28381  zyklon

[28382 rows x 1 columns]


In [9]:
coef_abs_lsrgest.shape

(10, 1)

In [26]:
l = pd.DataFrame(feature_mapping.loc[coef_abs_lsrgest.index, 'words'], columns=['words'])
print(l['words'].values)

['space' 'god' 'atheism' 'atheists' 'moon' 'sky' 'religion' 'bible'
 'keith' 'nick']
